# Assignment 2 - Building a Random Forest

This is a skeleton of a Random Forest classifier for the example data set in `data/example.csv`.

In [1]:
import csv
import math
from statistics import median, mode, mean
from collections import Counter
from enum import Enum
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

Some simple type definitions.

In [2]:
class AttrType(Enum):
    cat = 0  # categorical (qualitative) attribute
    num = 1  # numerical (quantitative) attribute
    target = 2  # target label


class NodeType(Enum):
    root = 0
    internal = 1
    leaf = 2


class SplitType(Enum):
    bin = 0  # binary split
    multi = 1  # multi-way split

Also, some basic classes to represent an attribute, a spltting procedure, and a node.

In [3]:
class Attribute(object):
    def __init__(self, label, type):
        assert type in AttrType
        self.label = label
        self.type = type
        self.stat = None  # holds mean for numerical and mode for categorical attributes


class Splitting(object):
    def __init__(self, attr, infogain, split_type, cond, splits):
        self.attr = attr  # attribute ID (index in ATTR)
        self.infogain = infogain  # information gain if splitting is done on this attribute
        self.split_type = split_type  # one of SplitType
        self.cond = cond  # splitting condition, i.e., values on outgoing edges
        self.splits = splits  # list of training records (IDs) for each slitting condition


class Node(object):
    def __init__(self, id, type, parent_id, children=None, edge_value=None, val=None, split_type=None, split_cond=None,
                 infogain=None):
        self.id = id  # ID (same as the index in DT.model list)
        self.type = type  # one of NodeType
        self.parent_id = parent_id  # ID of parent node (None if root)
        self.children = children  # list of IDs of child nodes
        self.edge_value = edge_value  # the value of the incoming edge (only if not root node)
        self.val = val  # if root or internal node: the attribute that is compared at that node; if leaf node: the target value
        self.split_type = split_type  # one of SplitType
        self.split_cond = split_cond  # splitting condition (median value for binary splits on numerical values; otherwise a list of categorical values (corresponding to child nodes))
        self.infogain = infogain

    def append_child(self, node_id):
        self.children.append(node_id)

The input filename is hard-coded.

In [4]:
INFILE = "data/housing_price_train.csv"
test = "data/housing_price_test.csv"

The attribute labels types are hard-coded too (the same order as in the file!).

In [24]:
ATTR = [Attribute("0", AttrType.num), Attribute("1", AttrType.num),
        Attribute("2", AttrType.num), 
        Attribute("3", AttrType.num), Attribute("4", AttrType.num),
        Attribute("target", AttrType.target)]

The index of the target attribute (assuming it's the last).

In [25]:
IDX_TARGET = len(ATTR) - 1 

### Pre-processing
Here I will do the required pre-processing steps

In [26]:
data = pd.read_csv(INFILE)
test_data = pd.read_csv(test, index_col ="Id")

In [27]:
test_index = test_data.index

Int64Index([1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470,
            ...
            3011, 3012, 3013, 3014, 3015, 3016, 3017, 3018, 3019, 3020],
           dtype='int64', name='Id', length=1560)

In [28]:
data.replace(to_replace=["?", "none", "None"], value=np.nan, inplace=True)
data.dropna(thresh=1400, inplace=True, axis=1)
c = data.columns
test_data1 = test_data.filter(c)

null_columns=test_data1.columns[test_data1.isna().any()]
test_data1[null_columns].isna().sum()
null_columns2=data.columns[data.isnull().any()]
data[null_columns2].isnull().sum()

MSZoning         4
Utilities        2
Exterior1st      1
Exterior2nd      1
MasVnrArea      15
BsmtQual        45
BsmtCond        46
BsmtExposure    45
BsmtFinType1    43
BsmtFinSF1       1
BsmtFinType2    43
BsmtFinSF2       1
BsmtUnfSF        1
TotalBsmtSF      1
BsmtFullBath     2
BsmtHalfBath     2
KitchenQual      1
Functional       2
GarageCars       1
GarageArea       1
SaleType         1
dtype: int64

In [29]:
data["MasVnrArea"].value_counts()
data["BsmtQual"].value_counts()
data["BsmtCond"].value_counts()
data["BsmtExposure"].value_counts()
data["BsmtFinType1"].value_counts()
data["BsmtFinType2"].value_counts()
data["Electrical"].value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [30]:
data["MasVnrArea"].replace(to_replace=np.nan, value=0.0, inplace=True)
data["BsmtQual"].replace(to_replace=np.nan, value="TA", inplace=True)
data["BsmtCond"].replace(to_replace=np.nan, value="TA", inplace=True)
data["BsmtExposure"].replace(to_replace=np.nan, value="No", inplace=True)
data["BsmtFinType1"].replace(to_replace=np.nan, value="Unf", inplace=True)
data["BsmtFinType2"].replace(to_replace=np.nan, value="Unf", inplace=True)
data["Electrical"].replace(to_replace=np.nan, value="SBrkr", inplace=True)

In [31]:
test_data1["MSZoning"].value_counts()
test_data1["Utilities"].value_counts()
test_data1["Exterior1st"].value_counts()
test_data1["Exterior2nd"].value_counts()
test_data1["MasVnrArea"].value_counts()
test_data1["BsmtQual"].value_counts()
test_data1["BsmtCond"].value_counts()
test_data1["BsmtExposure"].value_counts()
test_data1["BsmtFinType1"].value_counts()
test_data1["BsmtFinSF1"].value_counts()
test_data1["BsmtFinType2"].value_counts()
test_data1["BsmtFinSF2"].value_counts()
test_data1["BsmtUnfSF"].value_counts()
test_data1["TotalBsmtSF"].value_counts()
test_data1["BsmtFullBath"].value_counts()
test_data1["BsmtHalfBath"].value_counts()
test_data1["KitchenQual"].value_counts()
test_data1["Functional"].value_counts()
test_data1["GarageCars"].value_counts()
test_data1["GarageArea"].value_counts()
test_data1["SaleType"].value_counts()

WD       1344
New       127
COD        47
ConLD      18
CWD         9
ConLI       4
Oth         4
ConLw       3
Con         3
Name: SaleType, dtype: int64

In [32]:
test_data1["MSZoning"].replace(to_replace=np.nan, value="RL", inplace=True)
test_data1["MasVnrArea"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["Utilities"].replace(to_replace=np.nan, value="AllPub", inplace=True)
test_data1["Exterior1st"].replace(to_replace=np.nan, value="VinylSd", inplace=True)
test_data1["Exterior2nd"].replace(to_replace=np.nan, value="VinylSd", inplace=True)
test_data1["BsmtQual"].replace(to_replace=np.nan, value="TA", inplace=True)
test_data1["BsmtCond"].replace(to_replace=np.nan, value="TA", inplace=True)
test_data1["BsmtExposure"].replace(to_replace=np.nan, value="No", inplace=True)
test_data1["BsmtFinType1"].replace(to_replace=np.nan, value="GLQ", inplace=True)
test_data1["BsmtFinSF1"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["BsmtFinType2"].replace(to_replace=np.nan, value="Unf", inplace=True)
test_data1["BsmtFinSF2"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["BsmtUnfSF"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["TotalBsmtSF"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["BsmtFullBath"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["BsmtHalfBath"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["KitchenQual"].replace(to_replace=np.nan, value="TA", inplace=True)
test_data1["Functional"].replace(to_replace=np.nan, value="Typ", inplace=True)
test_data1["GarageCars"].replace(to_replace=np.nan, value=2.0, inplace=True)
test_data1["GarageArea"].replace(to_replace=np.nan, value=0.0, inplace=True)
test_data1["SaleType"].replace(to_replace=np.nan, value="WD", inplace=True)

In [33]:
train_y = data.iloc[:,-1]
train_x = data.drop(["SalePrice", "Id"], axis=1)

In [34]:
scaler = preprocessing.StandardScaler()
scaler.fit(pd.get_dummies(train_x))
train_x_s = pd.DataFrame(scaler.transform(pd.get_dummies(train_x)))

In [35]:
scaler2 = preprocessing.StandardScaler()
scaler2.fit(pd.get_dummies(test_data1))
test_s = pd.DataFrame(scaler2.transform(pd.get_dummies(test_data1)))

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1200,20,RL,9353,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,7,2006,Oth,Abnorml
1201,60,RL,10400,Pave,Reg,Lvl,AllPub,Corner,Gtl,CollgCr,...,36,0,0,0,0,0,3,2009,WD,Normal
1202,50,RM,6000,Pave,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,...,0,208,0,0,0,0,5,2009,WD,Normal
1203,20,RL,9750,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,234,0,0,0,0,0,10,2009,WD,Normal
1204,20,RL,10140,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,88,0,0,0,0,0,7,2006,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,112,0,0,0,0,4,2010,WD,Normal


In [36]:
pca = PCA(n_components=5)
pca_df = pd.DataFrame(pca.fit_transform(pd.get_dummies(train_x_s)))

pca_test = pd.DataFrame(pca.fit_transform(pd.get_dummies(test_s)))
pca_df.insert(5, "target", train_y)

(1460, 245)


,0,1,2,3,4,target
0,3.901308,-1.914484,-1.759950,-1.935222,0.841347,208500
1,-0.532160,3.003882,-0.735380,-0.003686,-0.075656,181500
2,4.509899,-1.248840,-1.207175,-1.910417,1.100195,223500
3,-1.354403,-0.683796,1.994443,-1.860763,-0.941088,140000
4,6.244765,-0.783322,0.634022,-3.349012,0.624330,250000


A class DT representing the decision tree classifier. It could represent with methods:

  - a given impurity measure;
  - the search for the best attribute to split with;
  - the addition of a node to the tree;
  - a convenient model printer;
  - the recursive call for obtaining a tree;
  - a builder and an applier.

In [38]:
class DT(object):
    def __init__(self, dataframe):
        self.data = dataframe.reset_index(drop=True)  # training data set (loaded into memory)
        self.model = []  # decision tree model
        self.default_target = 0.0  # default target class


    def __find_best_attr(self, attrs, records):
        """
        Finds the attribute with the largest gain.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :return:
        """
        splittings = []  # holds the splitting information for each attribute

        for a in attrs:
            assert ATTR[a].type in AttrType
            splits = {}  # record IDs corresponding to each split
            # splitting condition depends on the attribute type
            if ATTR[a].type == AttrType.target:  # skip target attribute
                continue
            elif ATTR[a].type == AttrType.num:  # numerical attribute => binary split on median value
                split_mode = SplitType.bin
                temp = self.data.loc[self.data.index.isin(records)]
                split_cond = np.mean(temp[a])  # (i.e., if less or equal than this value)
                list1 = temp.loc[temp[a] >= split_cond].index
                list2 = temp.loc[temp[a] < split_cond].index
                s = {
                    "right":list(list1),
                    "left": list(list2)
                }
                # TODO collect training records for each split (in `splits`)
                splits[a] = s
               
            # TODO compute gain for attribute a
            tempLeft = self.data.loc[self.data.index.isin(splits[a]["left"])]
            tempRight = self.data.loc[self.data.index.isin(splits[a]["right"])]

            msee = len(records) * np.var(self.data["target"].values) - (len(tempLeft) * np.var(tempLeft["target"].values) + len(tempRight) * np.var(tempRight["target"].values))
            splitting = Splitting(a, msee, split_mode, split_cond, splits[a])
            splittings.append(splitting)
        
        # find best splitting
        best_splitting = sorted(splittings, key=lambda x: x.infogain, reverse=True)[0]
        return best_splitting

    def __add_node(self, parent_id, node_type=NodeType.internal, edge_value=None, val=None, split_type=None,
                   split_cond=None):
        """
        Adds a node to the decision tree.

        :param parent_id:
        :param node_type:
        :param edge_value:
        :param val:
        :param split_type:
        :param split_cond:
        :return:
        """
        node_id = len(self.model)  # id of the newly assigned node
        if not self.model:  # the tree is empty
            node_type = NodeType.root

        node = Node(node_id, node_type, parent_id, children=[], edge_value=edge_value, val=val, split_type=split_type,
                    split_cond=split_cond)
        self.model.append(node)

        # also add it as a child of the parent node
        if parent_id is not None:
            self.model[parent_id].append_child(node_id)

        return node_id

    def __id3(self, attrs, records, parent_id=None, value=None):
        """
        Function ID3 that returns a decision tree.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :param parent_id: ID of parent node
        :param value: Value corresponding to the parent attribute, i.e., label of the edge on which we arrived to this node
        :return:
        """
        # empty training set or empty set of attributes => create leaf node with default class
        if len(records) < 10 or not attrs:
            temp = self.data.loc[self.data.index.isin(records)]
            val = np.mean(temp["target"].values)
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=val)
            return

        # find the attribute with the largest gain
        splitting = self.__find_best_attr(attrs, records)
        # add node
        node_id = self.__add_node(parent_id, edge_value=value, val=splitting.attr, split_type=splitting.split_type,
                                  split_cond=splitting.cond)
        # TODO call tree construction recursively for each split
        self.__id3(attrs, list(splitting.splits["right"]), node_id, "right")
        self.__id3(attrs, list(splitting.splits["left"]), node_id, "left")

    def print_model(self, node_id=0, level=0):
        node = self.model[node_id]
        indent = "  " * level
        if node.type == NodeType.leaf:
            print(indent + str(node.edge_value) + " [Leaf node] class=" + str(node.val))
        else:
            cond = " <= " + str(node.split_cond) if ATTR[node.val].type == AttrType.num else " == ? "
            if node.type == NodeType.root:
                print("[Root node] '" + ATTR[node.val].label + "'" + cond)
            else:
                print(indent + str(node.edge_value) + " [Internal node] '" + ATTR[node.val].label + "'" + cond)
            # print tree for child notes recursively
            for n_id in node.children:
                self.print_model(n_id, level + 1)

    def build_model(self):
        self.model = []  # holds the decision tree model, represented as a list of nodes
        # Get majority class
        #   Note: Counter returns a dictionary, most_common(x) returns a list with the x most common elements as
        #         (key, count) tuples; we need to take the first element of the list and the first element of the tuple
        self.default_target = mean(self.data.target)
        self.__id3(set(range(len(ATTR) - 1)), list(range(len(self.data))))

    def __apply_model(self, record):
        node = self.model[0]
        while node.type != NodeType.leaf:
        	# TODO based on the value of the record's attribute that is tested in `node`,
        	# set `node` to one of its child nodes until a leaf node is reached
            child = None
            if (record[node.val] > float(node.split_cond)):
                child = "right"
            else:
                child = "left"
            for nchild in node.children:
                if self.model[nchild].edge_value == child:
                    node = self.model[nchild]
        return node.val
    
    def predict(self, records):
        predictions = []
        pred_val = self.__apply_model(records)
        # TODO append pred_val to predictions
        predictions.append(pred_val)
        return np.mean(predictions)
    
    

A class RF representing the random forest classifier. It could represent with methods:

  - subsampling from the dataset
  - build a model with multiple decision trees
  - apply the model

In [39]:
class RF(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.trees = []  # decision trees
        
    def __subsampling(self, train_set, sample_size_ratio):
        sample_number = round(len(train_set) * sample_size_ratio)
        # TODO: generate a subsample with replacement
        return train_set.sample(sample_number, replace=True)
        
    def build_model(self, train_set, number_of_trees):
        for i in range(number_of_trees):
            sample = self.__subsampling(train_set, 0.5)
            tree = DT(sample) # build a tree with sample data and split conditions
            tree.build_model()
            self.trees.append(tree)
            
    def predict(self, test_set):
        rf_predictions = []
        for row in test_set:
            predictions = [tree.predict(row) for tree in self.trees]
            rf_predictions.append(np.mean(predictions))
        return rf_predictions

In [40]:
def createSubmission(test_ids, predictions):
    sub = pd.DataFrame()
    sub['Id'] = test_ids
    sub['SalePrice'] = predictions
    sub.to_csv('submission.csv',index=False)

Finally, the main function building a random forest model, printing it and applying it on some unseen records.

In [41]:
def main():

    rf = RF()
    print("Build model:")
    rf.build_model(pca_df,5)
    
    pred = rf.predict(pca_test.values)
    createSubmission(test_index, pred)

if __name__ == "__main__":
    main()

Build model:
233
[Root node] '0' <= 0.1910855289733598
  right [Internal node] '2' <= 0.036641252844415814
    right [Internal node] '0' <= 4.555522685356231
      right [Internal node] '2' <= 2.61454953779365
        right [Internal node] '3' <= 2.439886299209125
          right [Internal node] '2' <= 5.7015805073404655
            right [Leaf node] class=243250.0
            left [Internal node] '4' <= -1.1178534531939146
              right [Leaf node] class=384963.0
              left [Leaf node] class=433983.5714285714
          left [Internal node] '2' <= 5.012596096384747
            right [Leaf node] class=600750.0
            left [Leaf node] class=384346.8333333333
        left [Internal node] '2' <= 1.2969240640766615
          right [Internal node] '4' <= -1.4569508861562865
            right [Internal node] '2' <= 1.9394431134426557
              right [Leaf node] class=380833.3333333333
              left [Leaf node] class=330521.0
            left [Internal node] '3' <= 